## Preprocessing 

### Library

In [1]:
import cv2
import numpy as np

import glob
import os

from pprint import pprint as pp

path_in = 'input/*'
path_out = 'out'
window_name = 'crop'
size_max_image = 500
debug_mode = True

### Function Images Cropping

In [2]:
def get_image_width_height(image):
    image_width = image.shape[1]  # current image's width
    image_height = image.shape[0]  # current image's height
    return image_width, image_height


def calculate_scaled_dimension(scale, image):
    # http://www.pyimagesearch.com/2014/01/20/basic-image-manipulations-in-python-and-opencv-resizing-scaling-rotating-and-cropping/
    image_width, image_height = get_image_width_height(image)
    ratio_of_new_with_to_old = scale / image_width
    dimension = (scale, int(image_height * ratio_of_new_with_to_old))
    return dimension


# def rotate_image(image, degree=180):
#     # http://www.pyimagesearch.com/2014/01/20/basic-image-manipulations-in-python-and-opencv-resizing-scaling-rotating-and-cropping/
#     image_width, image_height = get_image_width_height(image)
#     center = (image_width / 2, image_height / 2)
#     M = cv2.getRotationMatrix2D(center, degree, 1.0)
#     image_rotated = cv2.warpAffine(image, M, (image_width, image_height))
#     return image_rotated


def scale_image(image, size):
    image_resized_scaled = cv2.resize(
        image,
        calculate_scaled_dimension(
            size,
            image
        ),
        interpolation=cv2.INTER_AREA
    )
    return image_resized_scaled

def detect_box(image, cropIt, no):
    # https://stackoverflow.com/questions/36982736/how-to-crop-biggest-rectangle-out-of-an-image/36988763
    # Transform colorspace to YUV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    red_min = np.array([30, 1, 1], np.uint8)
    red_max = np.array([255, 255, 255], np.uint8)
    mask_inverse = cv2.inRange(hsv, red_min, red_max)
    mask = cv2.bitwise_not(mask_inverse)

    # Blur to filter high frequency noises
    image_blurred = cv2.GaussianBlur(mask, (3, 3), 0)
#     if (debug_mode):  show_image(image_blurred, window_name)

    # Apply canny edge-detector
    edges = cv2.Canny(image_blurred, 0, 300, apertureSize=3)
#     if (debug_mode): show_image(edges, window_name)

    # Find extrem outer contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    

    # https://stackoverflow.com/questions/37803903/opencv-and-python-for-auto-cropping
    # Remove large countours
    new_contours = []
    for c in contours:
        if cv2.contourArea(c) < 4000000:
            new_contours.append(c)

    # Get overall bounding box
    best_box = [-1, -1, -1, -1]
    cpt = 0
    for c in new_contours:
        rect = cv2.boundingRect(c)
        
        #cropping the image to the value of the bounding rectangle
        img2=image[rect[1]:rect[1]+rect[3],rect[0]:rect[0]+rect[2]]
        width, height = img2.shape[:2]
        if (width>16 and height >16):
            cv2.imwrite("crop/biji_"+str(no)+"."+str(cpt)+".png", img2)
            cpt=cpt+1;
        
    
    if (debug_mode):
         #                                      b  g   r
        cv2.drawContours(image, contours, -1, (0, 255, 0), 1)
#         show_image(image, window_name)
            
    if (debug_mode):
        cv2.rectangle(image, (rect[0], rect[1]), (rect[2], rect[3]), (0, 255, 0), 1)
        #show_image(image, window_name)

    if (cropIt):
        image = image[rect[1]:rect[3], rect[0]:rect[2]]
        if (debug_mode): show_image(image, window_name)

    return image


def show_image(image, window_name):
    # Show image
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    cv2.imshow(window_name, image)
    image_width, image_height = get_image_width_height(image)
    cv2.resizeWindow(window_name, image_width, image_height)

    # Wait before closing
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def cut_of_top(image, pixel):
    image_width, image_height = get_image_width_height(image)

    # startY, endY, startX, endX coordinates
    new_y = 0+pixel
    image = image[new_y:image_height, 0:image_width]
    return image

def cut_of_bottom(image, pixel):
    image_width, image_height = get_image_width_height(image)

    # startY, endY, startX, endX coordinates
    new_height = image_height-pixel
    image = image[0:new_height, 0:image_width]
    return image

def cut_of_width(image, pixel):
    image_width, image_height = get_image_width_height(image)

    # startY, endY, startX, endX coordinates
    new_width = 0+pixel
    image = image[0:image_height, new_width:image_width]
    return image

### Main Preprocessing 

In [3]:
no = 0
for file_iterator in glob.iglob(path_in):
    image = cv2.imread(file_iterator)

    #image = rotate_image(image)
    image = cut_of_top(image, 1000)
    image = cut_of_bottom(image, 750)
    image = cut_of_width(image, 1200)
    
    image = scale_image(image, size_max_image)
    if (debug_mode): show_image(image, window_name)

    image = detect_box(image, False, no)

    # Create out path
    if not os.path.exists(path_out):
        os.makedirs(path_out)

    # Build output file path
    file_name_ext = os.path.basename(file_iterator)
    file_name, file_extension = os.path.splitext(file_name_ext)
    file_path = os.path.join(path_out, file_name + '.cropped' + file_extension)

    # Write out file
    cv2.imwrite(file_path, image)

    print("Transform file {} to {}".format(
        file_iterator,
        file_path
    ))
    
    no = no +1

Transform file input\baik1.jpg to out\baik1.cropped.jpg
Transform file input\baik10.jpg to out\baik10.cropped.jpg
Transform file input\baik2.jpg to out\baik2.cropped.jpg
Transform file input\baik3.jpg to out\baik3.cropped.jpg
Transform file input\baik4.jpg to out\baik4.cropped.jpg
Transform file input\baik5.jpg to out\baik5.cropped.jpg
Transform file input\baik6.jpg to out\baik6.cropped.jpg
Transform file input\baik7.jpg to out\baik7.cropped.jpg
Transform file input\baik8.jpg to out\baik8.cropped.jpg
Transform file input\baik9.jpg to out\baik9.cropped.jpg
Transform file input\jelek1.jpg to out\jelek1.cropped.jpg
Transform file input\jelek10.jpg to out\jelek10.cropped.jpg
Transform file input\jelek2.jpg to out\jelek2.cropped.jpg
Transform file input\jelek3.jpg to out\jelek3.cropped.jpg
Transform file input\jelek4.jpg to out\jelek4.cropped.jpg
Transform file input\jelek5.jpg to out\jelek5.cropped.jpg
Transform file input\jelek6.jpg to out\jelek6.cropped.jpg
Transform file input\jelek7.jp